<font color="#CA3532"><h1 align="left">Valoración de Inmuebles</h1></font>
<font color="#6E6E6E"><h2 align="left">Auditoría, limpieza y preprocesado de datos</h2></font>

#### Manuel Sánchez-Montañés

Primero importamos las bibliotecas que necesitaremos. Además utilizamos la primera celda de código para activar el modo *inline* para los gráficos generados por *matplotlib*. También iniciamos la semilla del generador aleatorio.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 500) # para que al visualizar imprima todas

In [ ]:
startbold = '\033[1m'
endbold = '\033[0m'

# Problema de tasación de casas en Ames

Ahora utilizaremos un conjunto de datos que describe la venta de propiedades residenciales individuales en Ames (Iowa, USA) desde 2006 hasta 2010. El conjunto de entrenamiento contiene 1460 observaciones y un gran número de variables explicativas (23 nominales, 23 ordinales, 14 discretas y 20 continuas) involucradas en la evaluación del valor de las viviendas. Para más detalle ver el archivo **data_description.txt**.

In [ ]:
df = pd.read_csv('./data/dataset_housing.csv')
df.head().T

# Limpieza y Análisis Exploratorio de Datos (EDA)

### Tipos de datos

In [ ]:
pd.DataFrame(df.dtypes, columns=["Tipo"])

### Visión global de los missing values

In [ ]:
print("Número de casas:", len(df))
print()
print("Variables en las que hay missing values (y número de ellos):")
aux = df.isna().sum()
pd.DataFrame(aux[aux!=0], columns=["Número de missings"])

## Gestión de cada columna

La información de cada variable la tomo del fichero **data_description.txt**

### 1. Columna Id

In [ ]:
len(df)

In [ ]:
len(np.unique(df["Id"]))

Es un identificador único de cada casa. La elimino.

In [ ]:
del df["Id"]

### 2. Columna MSSubClass

**Nominal.** Identifies the type of dwelling involved in the sale.	

        20	1-STORY 1946 & NEWER ALL STYLES
        30	1-STORY 1945 & OLDER
        40	1-STORY W/FINISHED ATTIC ALL AGES
        45	1-1/2 STORY - UNFINISHED ALL AGES
        50	1-1/2 STORY FINISHED ALL AGES
        60	2-STORY 1946 & NEWER
        70	2-STORY 1945 & OLDER
        75	2-1/2 STORY ALL AGES
        80	SPLIT OR MULTI-LEVEL
        85	SPLIT FOYER
        90	DUPLEX - ALL STYLES AND AGES
       120	1-STORY PUD (Planned Unit Development) - 1946 & NEWER
       150	1-1/2 STORY PUD - ALL AGES
       160	2-STORY PUD - 1946 & NEWER
       180	PUD - MULTILEVEL - INCL SPLIT LEV/FOYER
       190	2 FAMILY CONVERSION - ALL STYLES AND AGES

In [ ]:
print("Tipo:", df["MSSubClass"].dtypes)

In [ ]:
print("Número de missing values:", df["MSSubClass"].isnull().sum())
print("Valores únicos:", np.unique(df["MSSubClass"]))

Como es nominal, la paso a string para que sea procesada por get_dummies que haré al final:

In [ ]:
df["MSSubClass"] = df["MSSubClass"].astype(str)
print("Tipo:", df["MSSubClass"].dtypes)

In [ ]:
print("Valores únicos:", np.unique(df["MSSubClass"]))

### 3. Columna MSZoning

**Nominal**. Identifies the general zoning classification of the sale.
		
       A	Agriculture
       C	Commercial
       FV	Floating Village Residential
       I	Industrial
       RH	Residential High Density
       RL	Residential Low Density
       RP	Residential Low Density Park 
       RM	Residential Medium Density

In [ ]:
print("Número de missing values:", df["MSZoning"].isnull().sum())
print("Valores únicos:", np.unique(df["MSZoning"]))

Conclusión variable MSZoning: la dejo como categórica para después aplicar get_dummies sobre ella

### 4. Columna LotFrontage

**Numérica**. LotFrontage: Linear feet of street connected to property

In [ ]:
flags = df["LotFrontage"].isna()
print("Número de missing en LotFrontage:", sum(flags))
print("Valores únicos no missing:", np.unique(df[~flags]["LotFrontage"]))

Conclusión: entiendo que los valores missing son porque la propiedad no da directamente a la calle. Por tanto los sustituyo por 0.

In [ ]:
df["LotFrontage"].fillna(0,inplace=True)

In [ ]:
df["LotFrontage"].isna().sum()

### 5. Columna LotArea

**Numérica.** LotArea: Lot size in square feet

In [ ]:
print("Número de missing values en LotArea:", df["LotArea"].isnull().sum())
print("Mínimo y máximo:", df["LotArea"].min(), "y", df["LotArea"].max())
plt.figure(figsize=(10,3))
plt.subplot(1,2,1)
sns.distplot(df["LotArea"], rug=True)
plt.subplot(1,2,2)
sns.distplot(np.log(df["LotArea"]), rug=True)
plt.xlabel("Log LotArea");

Conclusión: LotArea parece razonable, su distribución no es simétrica y tiene una cola larga. Convendría quitar outliers (puntos mayor que 100000) o usar Log LotArea. Si esperamos que el valor del inmueble tienda a ser proporcional al de su superficie quizá no deberíamos transformar con el logaritmo.

In [ ]:
flags = df["LotArea"]>100000
df["MSZoning"][flags]

Opto por quitar esos cuatro outliers:

In [ ]:
df = df[~flags]

In [ ]:
np.unique(df["LotFrontage"])

### 6. Columna Street

**Nominal**. Type of road access to property

       Grvl	Gravel	
       Pave	Paved

In [ ]:
print("Número de missing values:", df["Street"].isnull().sum())
print("Valores únicos:", np.unique(df["Street"], return_counts=True))

Conclusión: no hay missing values, y en el fondo es una variable binaria. Pero todos los valores son 'Pave' menos cinco, por lo que no aporta prácticamente información. Por tanto la elimino.

In [ ]:
del df["Street"]

### 7. Columna Alley

**Nominal.** Type of alley access to property

       Grvl	Gravel
       Pave	Paved
       NA 	No alley access

In [ ]:
flags = df["Alley"].isnull()
flags.sum()

In [ ]:
np.unique(df["Alley"][~flags])

Conclusión: recodifico esta variable como 0 si no tiene este tipo de acceso, 1 si este acceso no está pavimentado y 2 si lo está.

In [ ]:
df.loc[flags,"Alley"] = 0
df[df["Alley"] == "Grvl"] = 1
df[df["Alley"] == "Pave"] = 2

In [ ]:
print(len(df))
np.unique(df["Alley"], return_counts=True)

### 8. Columna LotShape

**Ordinal.** General shape of property

       Reg	Regular	
       IR1	Slightly irregular
       IR2	Moderately Irregular
       IR3	Irregular

In [ ]:
print("Número de missing values:", df["LotShape"].isnull().sum())

In [ ]:
print(list(df["LotShape"])[:100])

In [ ]:
np.unique(df["LotShape"].astype(str), return_counts=True)

Vemos que aparecen los valores 1,2 pero no vienen en las especificaciones. Asumimos que son "IR1", "IR2":

In [ ]:
df.loc[df["LotShape"]==1,"LotShape"] = "IR1"
df.loc[df["LotShape"]==2,"LotShape"] = "IR2"

In [ ]:
np.unique(df["LotShape"])

A continuación realizo mi recodificación:

In [ ]:
df["LotShape"] = df["LotShape"].map({"Reg":0,"IR1":1,"IR2":2,"IR3":3})

In [ ]:
np.unique(df["LotShape"])

### 9. Columna LandContour

Flatness of the property

       Lvl	Near Flat/Level	
       Bnk	Banked - Quick and significant rise from street grade to building
       HLS	Hillside - Significant slope from side to side
       Low	Depression

**Considero que tiene sentido un orden y la trato como ordinal.**

In [ ]:
df["LandContour"]

Vemos que pasa parecido con la anterior (hay valores no especificados: 1,2)

In [ ]:
np.unique(df["LandContour"].astype(str), return_counts=True)

In [ ]:
print((df["LandContour"]==1).sum())
print((df["LandContour"]==2).sum())

Conclusión: no está claro cómo hacer corresponder los valores que no están en la especificación (1 y 2) con los que sí están. Elimino esos casos con valores 1 y 2:

In [ ]:
df = df[(df["LandContour"]!=1) & (df["LandContour"]!=2)]

In [ ]:
np.unique(df["LandContour"], return_counts=True)

Ahora recodifico como entero:

In [ ]:
df["LandContour"] = df["LandContour"].map({"Low":0,"HLS":1,"Bnk":2,"Lvl":3})

Pero al haber eliminado algunas casas ahora la columna Alley es constante:

In [ ]:
print(len(df))
np.unique(df["Alley"], return_counts=True)

Por lo que la elimino:

In [ ]:
del df["Alley"]

### 10. Columna Utilities

**Ordinal.** Type of utilities available
		
       AllPub	All public Utilities (E,G,W,& S)
       NoSewr	Electricity, Gas, and Water (Septic Tank)
       NoSeWa	Electricity and Gas Only
       ELO	Electricity only

In [ ]:
print("Número de missing values:", df["Utilities"].isnull().sum())
print("Valores únicos:", np.unique(df["Utilities"].astype(str), return_counts=True))

Vemos que todos los valores son 'AllPub' menos uno que es 'NoSeWa'. Conclusión: esta variable no aporta casi información, la eliminamos.

In [ ]:
del df["Utilities"]

### 11. Columna LotConfig

Lot configuration

       Inside	Inside lot
       Corner	Corner lot
       CulDSac	Cul-de-sac
       FR2	Frontage on 2 sides of property
       FR3	Frontage on 3 sides of property

**La voy a tratar como ordinal.**

In [ ]:
print("Número de missings:", df["LotConfig"].isnull().sum())
print("Valores únicos:", np.unique(df["LotConfig"], return_counts=True))

Ahora la recodifico como entero

In [ ]:
df["LotConfig"] = df["LotConfig"].map({"Inside":0,"Corner":1,"CulDSac":2,
                                       "FR2":3,"FR3":4})

### 12. Columna LandSlope

**Ordinal.** Slope of property
		
       Gtl	Gentle slope
       Mod	Moderate Slope	
       Sev	Severe Slope

In [ ]:
print("Número de missing values:", df["LandSlope"].isnull().sum())
print("Valores únicos:", np.unique(df["LandSlope"],return_counts=True))

Conclusión: la recodifico como 0, 1 o 2

In [ ]:
df["LandSlope"] = df["LandSlope"].map({"Gtl":0,"Mod":1,"Sev":2})

### 13. Columna Neighborhood

**Nominal.** Physical locations within Ames city limits

       Blmngtn	Bloomington Heights
       Blueste	Bluestem
       BrDale	Briardale
       BrkSide	Brookside
       ClearCr	Clear Creek
       CollgCr	College Creek
       Crawfor	Crawford
       Edwards	Edwards
       Gilbert	Gilbert
       IDOTRR	Iowa DOT and Rail Road
       MeadowV	Meadow Village
       Mitchel	Mitchell
       Names	North Ames
       NoRidge	Northridge
       NPkVill	Northpark Villa
       NridgHt	Northridge Heights
       NWAmes	Northwest Ames
       OldTown	Old Town
       SWISU	South & West of Iowa State University
       Sawyer	Sawyer
       SawyerW	Sawyer West
       Somerst	Somerset
       StoneBr	Stone Brook
       Timber	Timberland
       Veenker	Veenker

In [ ]:
print("Número de missings:", df["Neighborhood"].isnull().sum())
print("Valores únicos:", np.unique(df["Neighborhood"]))

Conclusión: la dejo como está (luego se hará un get_dummies sobre ella)

### 14. Columnas Condition1 y Condition2

**Columna Condition1:**

**Nominal.** Proximity to various conditions
	
       Artery	Adjacent to arterial street
       Feedr	Adjacent to feeder street	
       Norm	Normal	
       RRNn	Within 200' of North-South Railroad
       RRAn	Adjacent to North-South Railroad
       PosN	Near positive off-site feature--park, greenbelt, etc.
       PosA	Adjacent to postive off-site feature
       RRNe	Within 200' of East-West Railroad
       RRAe	Adjacent to East-West Railroad

**Columna Condition2:**

**Nominal.** Proximity to various conditions (if more than one is present)
		
       Artery	Adjacent to arterial street
       Feedr	Adjacent to feeder street	
       Norm	Normal	
       RRNn	Within 200' of North-South Railroad
       RRAn	Adjacent to North-South Railroad
       PosN	Near positive off-site feature--park, greenbelt, etc.
       PosA	Adjacent to postive off-site feature
       RRNe	Within 200' of East-West Railroad
       RRAe	Adjacent to East-West Railroad

In [ ]:
print("Número de missing values en Condition1:", df["Condition1"].isnull().sum())
print("Valores únicos:", np.unique(df["Condition1"], return_counts=True))

In [ ]:
print("Número de missing values en Condition2:", df["Condition2"].isnull().sum())
print("Valores únicos:", np.unique(df["Condition2"], return_counts=True))

In [ ]:
df[["Condition1","Condition2"]][:10]

In [ ]:
cond = pd.get_dummies(df["Condition1"])
cond[:3]

In [ ]:
cond2 = pd.get_dummies(df["Condition2"])
cond2[:3]

In [ ]:
cond[cond2.columns] = cond[cond2.columns] + cond2
cond[:5]

In [ ]:
cond.clip(upper=1, inplace=True)
cond[:5]

In [ ]:
cond.rename(columns={x:"Condition_"+x for x in cond.columns}, inplace=True)
cond[:3]

In [ ]:
sum(df.index != cond.index)

In [ ]:
df = df.join(cond)
df[:3]

Nos quedamos con las variables dummies creadas por lo que eliminamos las originales:

In [ ]:
del df["Condition1"]
del df["Condition2"]

### 15. Columna BldgType

**Nominal.** Type of dwelling
		
       1Fam	Single-family Detached	
       2FmCon	Two-family Conversion; originally built as one-family dwelling
       Duplx	Duplex
       TwnhsE	Townhouse End Unit
       TwnhsI	Townhouse Inside Unit

In [ ]:
print("Número de missing values:", df["BldgType"].isnull().sum())
print("Valores únicos:", np.unique(df["BldgType"], return_counts=True))

Conclusión: la dejo como está (para hacer posteriormente get_dummies sobre ella)

### 16. Columna HouseStyle

**Nominal.** Style of dwelling
	
       1Story	One story
       1.5Fin	One and one-half story: 2nd level finished
       1.5Unf	One and one-half story: 2nd level unfinished
       2Story	Two story
       2.5Fin	Two and one-half story: 2nd level finished
       2.5Unf	Two and one-half story: 2nd level unfinished
       SFoyer	Split Foyer
       SLvl	Split Level

In [ ]:
print("Número de missings:", df["HouseStyle"].isnull().sum())
print("Valores únicos:", np.unique(df["HouseStyle"]))

Conclusión: la dejo como está (para luego hacer get_dummies)

### 17. Columnas OverallQual y OverallCond

**Numéricas.**

**OverallQual:** Rates the overall material and finish of the house

**OverallCond:** Rates the overall condition of the house

       10	Very Excellent
       9	Excellent
       8	Very Good
       7	Good
       6	Above Average
       5	Average
       4	Below Average
       3	Fair
       2	Poor
       1	Very Poor

In [ ]:
print("Número de missings en OverallQual:", df["OverallQual"].isnull().sum())
print("Valores únicos:", np.unique(df["OverallQual"]))

In [ ]:
print("Número de missings en OverallCond:", df["OverallCond"].isnull().sum())
print("Valores únicos:", np.unique(df["OverallCond"]))

Conclusión: las dejo como están

### 18. Columnas YearBuilt y YearRemodAdd

**Numéricas.**

**YearBuilt:** Original construction date

**YearRemodAdd:** Remodel date (same as construction date if no remodeling or additions)

In [ ]:
print("Número de missings en YearBuilt:", df["YearBuilt"].isnull().sum())
print("Valores únicos:", np.unique(df["YearBuilt"]))

In [ ]:
print("Número de missings en YearRemodAdd:", df["YearRemodAdd"].isnull().sum())
print("Valores únicos:", np.unique(df["YearRemodAdd"]))

Conclusión: las dejo como están

### 19. Columna RoofStyle

**Nominal.** Type of roof

       Flat	Flat
       Gable	Gable
       Gambrel	Gabrel (Barn)
       Hip	Hip
       Mansard	Mansard
       Shed	Shed

In [ ]:
print("Número de missings:", df["RoofStyle"].isnull().sum())
print("Valores únicos:", np.unique(df["RoofStyle"], return_counts=True))

Conclusión: hay muy pocas casas cuyos valores no sean 'Gable' o 'Hip' (13+7+5+2=27 en total). Solución: o separo solo en dos categorías (Gable / no Gable) o en tres (Gable / Hip / Other). Opto por esto último:

In [ ]:
for x in ["Flat","Gambrel","Mansard","Shed"]:
    df.loc[df["RoofStyle"]==x, "RoofStyle"] = "Other"

In [ ]:
print("Número de missings:", df["RoofStyle"].isnull().sum())
print("Valores únicos:", np.unique(df["RoofStyle"], return_counts=True))

En caso de que la tercera categoría no sea informativa, esa variable será eliminada en el proceso de selección de variables.

### 20. Columna RoofMatl

**Nominal.** Roof material

       ClyTile	Clay or Tile
       CompShg	Standard (Composite) Shingle
       Membran	Membrane
       Metal	Metal
       Roll	Roll
       Tar&Grv	Gravel & Tar
       WdShake	Wood Shakes
       WdShngl	Wood Shingles

In [ ]:
print("Número de missings:", df["RoofMatl"].isnull().sum())
print("Valores únicos:", np.unique(df["RoofMatl"], return_counts=True))

Conclusión: en vista a las estadísticas, considero sólo dos clases: CompShg / No CompShg

In [ ]:
df["RoofMatl"] = 1*(df["RoofMatl"]=='CompShg')

# La renombro:
df.rename(columns={"RoofMatl":"RoofMatl_CompShg"}, inplace=True)

In [ ]:
print("Número de missings de RoofMatl_CompShg:", df["RoofMatl_CompShg"].isnull().sum())
print("Valores únicos:", np.unique(df["RoofMatl_CompShg"], return_counts=True))

### 21. Columnas Exterior1st y Exterior2nd

**Las dos columnas son nominales.**

**Variable Exterior1st:**

Exterior covering on house

       AsbShng	Asbestos Shingles
       AsphShn	Asphalt Shingles
       BrkComm	Brick Common
       BrkFace	Brick Face
       CBlock	Cinder Block
       CemntBd	Cement Board
       HdBoard	Hard Board
       ImStucc	Imitation Stucco
       MetalSd	Metal Siding
       Other	Other
       Plywood	Plywood
       PreCast	PreCast	
       Stone	Stone
       Stucco	Stucco
       VinylSd	Vinyl Siding
       Wd Sdng	Wood Siding
       WdShing	Wood Shingles

**Variable Exterior2nd:**

Exterior covering on house (if more than one material)

       AsbShng	Asbestos Shingles
       AsphShn	Asphalt Shingles
       BrkComm	Brick Common
       BrkFace	Brick Face
       CBlock	Cinder Block
       CemntBd	Cement Board
       HdBoard	Hard Board
       ImStucc	Imitation Stucco
       MetalSd	Metal Siding
       Other	Other
       Plywood	Plywood
       PreCast	PreCast
       Stone	Stone
       Stucco	Stucco
       VinylSd	Vinyl Siding
       Wd Sdng	Wood Siding
       WdShing	Wood Shingles

In [ ]:
print("Número de missing values en Exterior1st:", df["Exterior1st"].isnull().sum())
print("Valores únicos:", np.unique(df["Exterior1st"]))

In [ ]:
print("Número de missing values en Exterior2nd:", df["Exterior2nd"].isnull().sum())
print("Valores únicos:", np.unique(df["Exterior2nd"]))

Hay algunos valores que no son como en las especificaciones:
* **Exterior2nd:** Brk Cmn -> BrkComm
* **Exterior2nd:** CmentBd -> CemntBd
* **Exterior2nd:** Wd Shng -> WdShing

In [ ]:
df.loc[df["Exterior2nd"]=="Brk Cmn","Exterior2nd"] = "BrkComm"
df.loc[df["Exterior2nd"]=="CmentBd","Exterior2nd"] = "CemntBd"
df.loc[df["Exterior2nd"]=="Wd Shng","Exterior2nd"] = "WdShing"
print("Valores únicos de Exterior2nd:", np.unique(df["Exterior2nd"]))

In [ ]:
df[["Exterior1st","Exterior2nd"]][:10]

In [ ]:
exterior = pd.get_dummies(df["Exterior1st"])
exterior["Other"] = 0 # este valor lo tiene Exterior2nd pero no Exterior1st
exterior[:3]

In [ ]:
exterior2 = pd.get_dummies(df["Exterior2nd"])
exterior2[:3]

In [ ]:
exterior[exterior2.columns] = exterior[exterior2.columns] + exterior2
exterior.clip(upper=1, inplace=True)
exterior.rename(columns={x:"Exterior_"+x for x in exterior.columns}, inplace=True)
exterior[:3]

In [ ]:
sum(df.index != exterior.index)

In [ ]:
df = df.join(exterior)
df[:3]

In [ ]:
del df["Exterior1st"]
del df["Exterior2nd"]

### 22. Columna MasVnrType

**Nominal.** Masonry veneer type

       BrkCmn	Brick Common
       BrkFace	Brick Face
       CBlock	Cinder Block
       None	None
       Stone	Stone

In [ ]:
print("Número de missings:", df["MasVnrType"].isnull().sum())

Conclusión: quito esas pocas casas donde hay missing

In [ ]:
len(df)

In [ ]:
df.dropna(subset=["MasVnrType"],axis=0,inplace=True)

In [ ]:
len(df)

In [ ]:
print("Valores únicos:", np.unique(df["MasVnrType"], return_counts=True))

### 23. Columna MasVnrArea

**Numérica.** Masonry veneer area in square feet

In [ ]:
print("Número de missings:", df["MasVnrArea"].isnull().sum())
print("Valores únicos:", np.unique(df["MasVnrArea"]))

Conclusión: la dejo como está

### 24. Columnas ExterQual y ExterCond

**Ordinales.**

**ExterQual:** Evaluates the quality of the material on the exterior 

**ExterCond:** Evaluates the present condition of the material on the exterior

       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       Po	Poor

In [ ]:
print("Número de missings en ExterQual:", df["ExterQual"].isnull().sum())
print("Valores únicos:", np.unique(df["ExterQual"], return_counts=True))

Conclusión: la recodifico como entera. Nivel más bajo: 1 (Poor). Nivel más alto: 5 (Excellent)

In [ ]:
df["ExterQual"] = df["ExterQual"].map({"Po":1,"Fa":2,"TA":3,"Gd":4,"Ex":5})

In [ ]:
# La otra variable: ExterCond
print("Número de missings en ExterCond:", df["ExterCond"].isnull().sum())
print("Valores únicos:", np.unique(df["ExterCond"], return_counts=True))

Hago lo mismo que en la anterior:

In [ ]:
df["ExterCond"] = df["ExterCond"].map({"Po":1,"Fa":2,"TA":3,"Gd":4,"Ex":5})

### 25. Columna Foundation

**Nominal.** Type of foundation
		
       BrkTil	Brick & Tile
       CBlock	Cinder Block
       PConc	Poured Contrete	
       Slab	Slab
       Stone	Stone
       Wood	Wood

In [ ]:
print("Número de missings:", df["Foundation"].isnull().sum())
print("Valores únicos:", np.unique(df["Foundation"], return_counts=True))

Conclusión: agrupo las categorías menos frecuentes (Slab, Stone, Wood) en 'Other'

In [ ]:
for x in ["Slab","Stone","Wood"]:
    df.loc[df["Foundation"]==x, "Foundation"] = "Other"

In [ ]:
print("Número de missings:", df["Foundation"].isnull().sum())
print("Valores únicos:", np.unique(df["Foundation"], return_counts=True))

### 26. Columnas Basement

**TotalBsmtSF:** Total square feet of basement area. Numérica.

In [ ]:
print("Número de missings:", df["TotalBsmtSF"].isnull().sum())
print("Número de valores=0:", (df["TotalBsmtSF"]==0).sum())

Dejo esta variable como está. Interpreto además que si la superficie del sótano es 0 es que no hay sótano.

**BsmtQual:** Evaluates the height of the basement. Ordinal.

       Ex	Excellent (100+ inches)	
       Gd	Good (90-99 inches)
       TA	Typical (80-89 inches)
       Fa	Fair (70-79 inches)
       Po	Poor (<70 inches
       NA	No Basement

In [ ]:
print("Número de missings:", df["BsmtQual"].isnull().sum())

Sale el mismo número de casos que ceros en superficie del sótano. Chequeo si efectivamente son los mismos casos:

In [ ]:
print(sum((df["TotalBsmtSF"]==0) & df["BsmtQual"].isnull()))

Sí, son los mismos casos. Por tanto sustituyo esos missings por "NA"

In [ ]:
df["BsmtQual"].fillna("NA",inplace=True)
print("Valores únicos:", np.unique(df["BsmtQual"]))

Finalmente las recodifico con un entero:

In [ ]:
df["BsmtQual"] = df["BsmtQual"].map({"NA":0,"Po":1,"Fa":2,"TA":3,"Gd":4,"Ex":5})

**Variable BsmtCond:** Evaluates the general condition of the basement. Ordinal.

       Ex	Excellent
       Gd	Good
       TA	Typical - slight dampness allowed
       Fa	Fair - dampness or some cracking or settling
       Po	Poor - Severe cracking, settling, or wetness
       NA	No Basement

In [ ]:
print("Número de missings:", df["BsmtCond"].isnull().sum())
print("De ellos, TotalBsmtSF=0 en",
      sum((df["TotalBsmtSF"]==0) & df["BsmtCond"].isnull()), "casos")

Pasa lo mismo que en el anterior caso, por lo que hago lo mismo:

In [ ]:
df["BsmtCond"].fillna("NA",inplace=True)
print("Valores únicos:", np.unique(df["BsmtCond"]))
df["BsmtCond"] = df["BsmtCond"].map({"NA":0,"Po":1,"Fa":2,"TA":3,"Gd":4,"Ex":5})

**Variable BsmtExposure:** Refers to walkout or garden level walls. Ordinal.

       Gd	Good Exposure
       Av	Average Exposure (split levels or foyers typically score average or above)	
       Mn	Mimimum Exposure
       No	No Exposure
       NA	No Basement

In [ ]:
print("Número de missings en BsmtExposure:", df["BsmtExposure"].isnull().sum())
print("De ellos, TotalBsmtSF=0 en",
      sum((df["TotalBsmtSF"]==0) & df["BsmtExposure"].isnull()), "casos")

Hay un caso más de los 36 que se esperaban. Primero relleno con "NA" los casos donde TotalBsmtSF es 0:

In [ ]:
df.loc[df["TotalBsmtSF"]==0,"BsmtExposure"] = "NA"
print("Número de missings en BsmtExposure:", df["BsmtExposure"].isnull().sum())

Y el caso restante lo elimino:

In [ ]:
df.dropna(subset=["BsmtExposure"], axis=0, inplace=True)

In [ ]:
print("Valores únicos:", np.unique(df["BsmtExposure"]))

Ahora hago lo mismo que con **BsmtCond:**

In [ ]:
df["BsmtExposure"] = df["BsmtExposure"].map({"NA":0,"No":1,"Mn":2,"Av":3,"Gd":4})

**Variable BsmtFinType1:** Rating of basement finished area. Ordinal.

       GLQ	Good Living Quarters
       ALQ	Average Living Quarters
       BLQ	Below Average Living Quarters	
       Rec	Average Rec Room
       LwQ	Low Quality
       Unf	Unfinshed
       NA	No Basement

In [ ]:
print("Número de missings:", df["BsmtFinType1"].isnull().sum())
print("De ellos, TotalBsmtSF=0 en",
      sum((df["TotalBsmtSF"]==0) & df["BsmtFinType1"].isnull()), "casos")

Pasa lo mismo que anteriormente, por lo que hago algo similar:

In [ ]:
df["BsmtFinType1"].fillna("NA",inplace=True)
print("Valores únicos:", np.unique(df["BsmtFinType1"]))
df["BsmtFinType1"] = df["BsmtFinType1"].map({"NA":0,"Unf":1,"LwQ":2,"Rec":3,
                                             "BLQ":4,"ALQ":5,"GLQ":6})

**Variable BsmtFinSF1:** Type 1 finished square feet. Numérica.

In [ ]:
print("Número de missings:", df["BsmtFinSF1"].isnull().sum())
df[["BsmtFinSF1"]].describe().T

Conclusión: la dejo como está

**Variable BsmtFinType2:**

Rating of basement finished area (if multiple types)

       GLQ	Good Living Quarters
       ALQ	Average Living Quarters
       BLQ	Below Average Living Quarters	
       Rec	Average Rec Room
       LwQ	Low Quality
       Unf	Unfinshed
       NA	No Basement

In [ ]:
print("Número de missings:", df["BsmtFinType2"].isnull().sum())
print("De ellos, TotalBsmtSF=0 en",
      sum((df["TotalBsmtSF"]==0) & df["BsmtFinType2"].isnull()), "casos")

Hay un caso más de los 36 que se esperaban. Primero relleno con "NA" los casos donde TotalBsmtSF es 0:

In [ ]:
df.loc[df["TotalBsmtSF"]==0,"BsmtFinType2"] = "NA"
print("Número de missings en BsmtFinType2:", df["BsmtFinType2"].isnull().sum())

Y el caso restante lo elimino:

In [ ]:
df.dropna(subset=["BsmtFinType2"], axis=0, inplace=True)

In [ ]:
print("Valores únicos:", np.unique(df["BsmtFinType2"]))

Ahora recodifico como entero:

In [ ]:
df["BsmtFinType2"] = df["BsmtFinType2"].map({"NA":0,"Unf":1,"LwQ":2,"Rec":3,
                                             "BLQ":4,"ALQ":5,"GLQ":6})

**Variable BsmtFinSF2:** Type 2 finished square feet. Numérica.

In [ ]:
print("Número de missings:", df["BsmtFinSF2"].isnull().sum())
df[["BsmtFinSF2"]].describe().T

Conclusión: la dejo como está

**Variable BsmtUnfSF:** Unfinished square feet of basement area. Numérica.

In [ ]:
print("Número de missings:", df["BsmtUnfSF"].isnull().sum())
df[["BsmtUnfSF"]].describe().T

Conclusión: la dejo como está

### 27. Columna Heating

**Nominal.** Type of heating

       Floor	Floor Furnace
       GasA	Gas forced warm air furnace
       GasW	Gas hot water or steam heat
       Grav	Gravity furnace	
       OthW	Hot water or steam heat other than gas
       Wall	Wall furnace

In [ ]:
print("Número de missings:", df["Heating"].isnull().sum())
print("Valores únicos:", np.unique(df["Heating"], return_counts=True))

Conclusión: en vista a las estadísticas, considero sólo dos clases: GasA / No GasA

In [ ]:
df["Heating"] = 1*(df["Heating"]=='GasA')

# La renombro:
df.rename(columns={"Heating":"Heating_GasA"}, inplace=True)

In [ ]:
print("Número de missings de Heating_GasA:", df["Heating_GasA"].isnull().sum())
print("Valores únicos:", np.unique(df["Heating_GasA"], return_counts=True))

### 28. Columna HeatingQC

**Ordinal.** Heating quality and condition

       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       Po	Poor

In [ ]:
print("Número de missings:", df["HeatingQC"].isnull().sum())
print("Valores únicos:", np.unique(df["HeatingQC"]))

Conclusión: Recodifico con entero

In [ ]:
df["HeatingQC"] = df["HeatingQC"].map({"Po":1,"Fa":2,"TA":3,"Gd":4,"Ex":5})

### 29. Columna CentralAir

**Nominal.** Central air conditioning

       N	No
       Y	Yes

In [ ]:
print("Número de missings:", df["CentralAir"].isnull().sum())
print("Valores únicos:", np.unique(df["CentralAir"]))

Conclusión: Recodifico con entero

In [ ]:
df["CentralAir"] = df["CentralAir"].map({"N":0,"Y":1})

### 30. Columna Electrical

**La interpreto como ordinal (de mejor a peor).** Electrical system

       SBrkr	Standard Circuit Breakers & Romex
       FuseA	Fuse Box over 60 AMP and all Romex wiring (Average)	
       FuseF	60 AMP Fuse Box and mostly Romex wiring (Fair)
       FuseP	60 AMP Fuse Box and mostly knob & tube wiring (poor)
       Mix	Mixed

In [ ]:
print("Número de missings:", df["Electrical"].isnull().sum())

Quito la casa en la que hay ese missing:

In [ ]:
df.dropna(subset=["Electrical"],axis=0,inplace=True)

In [ ]:
print("Número de missings:", df["Electrical"].isnull().sum())
print("Valores únicos:", np.unique(df["Electrical"], return_counts=True))

Conclusión: Recodifico con entero

In [ ]:
df["Electrical"] = df["Electrical"].map({"Mix":1,"FuseP":2,"FuseF":3,"FuseA":4,"SBrkr":5})

### 31. Columnas Variable 1stFlrSF, 2ndFlrSF, LowQualFinSF, GrLivArea, BsmtFullBath, BsmtHalfBath, FullBath, HalfBath, BedroomAbvGr, KitchenAbvGr, TotRmsAbvGrd

**Todas son variables numéricas.**

**1stFlrSF:** First Floor square feet

**2ndFlrSF:** Second floor square feet

**LowQualFinSF:** Low quality finished square feet (all floors)

**GrLivArea:** Above grade (ground) living area square feet

**BsmtFullBath:** Basement full bathrooms

**BsmtHalfBath:** Basement half bathrooms

**FullBath:** Full bathrooms above grade

**HalfBath:** Half baths above grade

**BedroomAbvGr:** Bedrooms above grade (does NOT include basement bedrooms)

**KitchenAbvGr:** Kitchens above grade

**TotRmsAbvGrd:** Total rooms above grade (does not include bathrooms)

In [ ]:
variables = ["1stFlrSF", "2ndFlrSF", "LowQualFinSF", "GrLivArea",
             "BsmtFullBath","BsmtHalfBath","FullBath",
             "HalfBath","BedroomAbvGr","KitchenAbvGr","TotRmsAbvGrd"]

In [ ]:
print("Número de missings:\n")
print(df[variables].isnull().sum())

In [ ]:
df[variables].describe().T

Conclusión: las dejo como están

### 32. Columna KitchenQual

**Ordinal.** Kitchen quality

       Ex	Excellent
       Gd	Good
       TA	Typical/Average
       Fa	Fair
       Po	Poor

In [ ]:
print("Número de missings:", df["KitchenQual"].isnull().sum())
print("Valores únicos antes de pasar a entero:", np.unique(df["KitchenQual"],
                                                           return_counts=True))

Ahora mapeo estos valores a enteros:

In [ ]:
df["KitchenQual"] = df["KitchenQual"].map({"Po":1,"Fa":2,"TA":3,"Gd":4,"Ex":5})

### 33. Columna Functional

**Ordinal.** Home functionality (Assume typical unless deductions are warranted)

       Typ	Typical Functionality
       Min1	Minor Deductions 1
       Min2	Minor Deductions 2
       Mod	Moderate Deductions
       Maj1	Major Deductions 1
       Maj2	Major Deductions 2
       Sev	Severely Damaged
       Sal	Salvage only

In [ ]:
print("Número de missings:", df["Functional"].isnull().sum())
print("Valores únicos antes de pasar a entero:", np.unique(df["Functional"],
                                                           return_counts=True))

Ahora mapeo estos valores a enteros:

In [ ]:
df["Functional"] = df["Functional"].map({"Sal":1,"Sev":2,"Maj2":3,"Maj1":4,
                                         "Mod":5,"Min2":6,"Min1":7,"Typ":8})

### 34. Columnas Fireplaces

**Fireplaces:** Number of fireplaces. **Numérica.**

**FireplaceQu:** Fireplace quality. **Ordinal.**

       Ex	Excellent - Exceptional Masonry Fireplace
       Gd	Good - Masonry Fireplace in main level
       TA	Average - Prefabricated Fireplace in main living area or Masonry Fireplace in basement
       Fa	Fair - Prefabricated Fireplace in basement
       Po	Poor - Ben Franklin Stove
       NA	No Fireplace

In [ ]:
vars_fi = ["Fireplaces","FireplaceQu"]
df[vars_fi].isnull().sum()

In [ ]:
(df[vars_fi]==0).sum()

¿Coinciden los ceros de Fireplaces con los missings de FireplaceQu?

In [ ]:
sum((df["Fireplaces"] == 0) &
    df["FireplaceQu"].isnull())

Respuesta: sí

In [ ]:
df[["Fireplaces"]].describe().T

Conclusión: dejo "Fireplaces" como está

Ahora imputo el string "NA" a los missing de "FireplaceQu":

In [ ]:
df["FireplaceQu"].fillna("NA",inplace=True)
print("Valores únicos antes de pasar a entero:", np.unique(df["FireplaceQu"]))

Y ahora mapeo esos valores a enteros:

In [ ]:
df["FireplaceQu"] = df["FireplaceQu"].map({"NA":0,"Po":1,"Fa":2,"TA":3,"Gd":4,"Ex":5})

### 35. Columnas Garaje

* **GarageCars:** Size of garage in car capacity. Numérica.

* **GarageArea:** Size of garage in square feet. Numérica.

* **GarageQual, GarageCond:** Ordinales.

* **GarageType:** Nominal.

* **GarageYrBlt, GarageFinish:** Numéricas.

In [ ]:
vars_gar = ["GarageCars","GarageArea","GarageQual",
            "GarageCond","GarageType","GarageYrBlt","GarageFinish"]
df[vars_gar].isnull().sum()

In [ ]:
(df[vars_gar]==0).sum()

¿Coinciden los ceros de GarageCars y GarageArea con los missings de las otras variables garaje?

In [ ]:
sum((df["GarageCars"] == 0) &
    (df["GarageArea"] == 0) &
    df["GarageQual"].isnull() &
    df["GarageCond"].isnull() &
    df["GarageType"].isnull() &
    df["GarageYrBlt"].isnull() &
    df["GarageFinish"].isnull())

Respuesta: sí

In [ ]:
df[["GarageCars"]].describe().T

Dejo a "GarageCars" como está. Los valores 0 indican que no hay garaje.

In [ ]:
df[["GarageArea"]].describe().T

Dejo a "GarageArea" como está. Los valores 0 indican que no hay garaje.

**Variable GarageQual:**

Garage quality

       Ex	Excellent
       Gd	Good
       TA	Typical/Average
       Fa	Fair
       Po	Poor
       NA	No Garage

Imputo el string "NA" a los missing:

In [ ]:
df["GarageQual"].fillna("NA",inplace=True)
print("Valores únicos antes de pasar a entero:", np.unique(df["GarageQual"]))

Y ahora mapeo estos valores a enteros:

In [ ]:
df["GarageQual"] = df["GarageQual"].map({"NA":0,"Po":1,"Fa":2,
                                         "TA":3,"Gd":4,"Ex":5})

**Variable GarageCond:**

Garage condition

       Ex	Excellent
       Gd	Good
       TA	Typical/Average
       Fa	Fair
       Po	Poor
       NA	No Garage

Hago lo mismo que en la variable anterior:

In [ ]:
df["GarageCond"].fillna("NA",inplace=True)
print("Valores únicos antes de pasar a entero:", np.unique(df["GarageCond"]))
df["GarageCond"] = df["GarageCond"].map({"NA":0,"Po":1,"Fa":2,
                                         "TA":3,"Gd":4,"Ex":5})

**Variable GarageType:**

Garage location
		
       2Types	More than one type of garage
       Attchd	Attached to home
       Basment	Basement Garage
       BuiltIn	Built-In (Garage part of house - typically has room above garage)
       CarPort	Car Port
       Detchd	Detached from home
       NA	No Garage

Hago algo similar a antes:

In [ ]:
df["GarageType"].fillna("NA",inplace=True)
print("Valores únicos antes de pasar a entero:", np.unique(df["GarageType"]))
df["GarageType"] = df["GarageType"].map({"NA":0,"Detchd":1,"CarPort":2,"BuiltIn":3,
                                         "Basment":4,"Attchd":5,"2Types":6})

**Variable GarageYrBlt:**

Year garage was built

In [ ]:
print("Número de missings:", df["GarageYrBlt"].isnull().sum())
df[["GarageYrBlt"]].describe().T

In [ ]:
df["GarageYrBlt"].min()

Ahora hay un problema: ¿con qué valor debo imputar los missings de "GarageYrBlt" (es decir, casas donde no hay garaje)? La idea es que un garaje construido no hace mucho es un valor a tener en cuenta en la tasación de una casa.

Una idea es sustituir esta variable por otra que llamaremos "GarageYrBlt_gt_1990", que es un flag indicando si el garaje se construyó a partir de 1990 o no. Las casas sin garaje tendrán este flag a 0:

In [ ]:
df["GarageYrBlt"].fillna(df["GarageYrBlt"].min())
df["GarageYrBlt"] = 1*(df["GarageYrBlt"]>=1990)

In [ ]:
df.rename(columns={"GarageYrBlt":"GarageYrBlt_gt_1990"},inplace=True)

**Variable GarageFinish:**

Interior finish of the garage

       Fin	Finished
       RFn	Rough Finished	
       Unf	Unfinished
       NA	No Garage

Hago algo similar a GarageCond:

In [ ]:
df["GarageFinish"].fillna("NA",inplace=True)
print("Valores únicos antes de pasar a entero:", np.unique(df["GarageFinish"]))
df["GarageFinish"] = df["GarageFinish"].map({"NA":0,"Unf":1,"RFn":2,"Fin":3})

Veamos ahora cómo salen las estadísticas de las variables garaje:

In [ ]:
vars_gar = ["GarageCars","GarageArea","GarageQual",
            "GarageCond","GarageType","GarageYrBlt_gt_1990","GarageFinish"]
df[vars_gar].isnull().sum()

### 36. Columna PavedDrive

**Ordinal.** Paved driveway

       Y	Paved 
       P	Partial Pavement
       N	Dirt/Gravel

In [ ]:
print("Número de missings:", df["PavedDrive"].isnull().sum())
print("Valores únicos antes de pasar a entero:", np.unique(df["PavedDrive"],
                                                           return_counts=True))

Finalmente las recodifico con un entero:

In [ ]:
df["PavedDrive"] = df["PavedDrive"].map({"N":0,"P":1,"Y":2})

### 37. Columnas WoodDeckSF, OpenPorchSF, EnclosedPorch, 3SsnPorch, ScreenPorch

**Son numéricas.**

* **WoodDeckSF:** Wood deck area in square feet
* **OpenPorchSF:** Open porch area in square feet
* **EnclosedPorch:** Enclosed porch area in square feet
* **3SsnPorch:** Three season porch area in square feet
* **ScreenPorch:** Screen porch area in square feet

In [ ]:
variables = ["WoodDeckSF","OpenPorchSF","EnclosedPorch",
             "3SsnPorch","ScreenPorch"]

In [ ]:
print("Número de missings:\n")
print(df[variables].isnull().sum())

In [ ]:
df[variables].describe().T

Conclusión: las dejo como están

### 38. Columnas Piscina

* **PoolArea:** Pool area in square feet. Numérica.

* **PoolQC:** Pool quality. Ordinal.
		
       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       NA	No Pool

In [ ]:
vars_pool = ["PoolArea","PoolQC"]
df[vars_pool].isnull().sum()

In [ ]:
(df[vars_pool]==0).sum()

¿Coinciden los ceros de PoolArea con los missings de PoolQC?

In [ ]:
sum((df["PoolArea"] == 0) &
    df["PoolQC"].isnull())

Respuesta: sí

In [ ]:
df[["PoolArea"]].describe().T

Conclusión: dejo "PoolArea" como está. Por otra parte, imputo el string "NA" a los missing de "PoolQC":

In [ ]:
df["PoolQC"].fillna("NA",inplace=True)
print("Valores únicos antes de pasar a entero:", np.unique(df["PoolQC"]))

Y ahora mapeo estos valores a enteros:

In [ ]:
df["PoolQC"] = df["PoolQC"].map({"NA":0,"Fa":1,"TA":2,"Gd":3,"Ex":4})

### 39. Columna Fence (cercado)

**Ordinal.** Fence quality
		
       GdPrv	Good Privacy
       MnPrv	Minimum Privacy
       GdWo	Good Wood
       MnWw	Minimum Wood/Wire
       NA	No Fence

In [ ]:
print("Número de missings:", df["Fence"].isnull().sum())

Sustituyo esos missings por el string "NA":

In [ ]:
df["Fence"].fillna("NA",inplace=True)

In [ ]:
print("Valores únicos antes de pasar a entero:", np.unique(df["Fence"],
                                                           return_counts=True))

Finalmente las recodifico con un entero:

In [ ]:
df["Fence"] = df["Fence"].map({"NA":0,"MnWw":1,"GdWo":2,"MnPrv":3,"GdPrv":4})

### 40. Variables de futuro

**Variable MoSold:**

Month Sold (MM)

In [ ]:
print("Número de missings:", df["MoSold"].isnull().sum())
print("Número de valores a cero:", sum(df["MoSold"]==0))
df[["MoSold"]].describe().T

Conclusión: variable de futuro, la quito

In [ ]:
del df["MoSold"]

**Variable YrSold:**

Year Sold (YYYY)

In [ ]:
print("Número de missings:", df["YrSold"].isnull().sum())
print("Número de valores a cero:", sum(df["YrSold"]==0))
df[["YrSold"]].describe().T

Conclusión: variable de futuro, la quito

In [ ]:
del df["YrSold"]

**Variable SaleType:**

Type of sale

       WD 	Warranty Deed - Conventional
       CWD	Warranty Deed - Cash
       VWD	Warranty Deed - VA Loan
       New	Home just constructed and sold
       COD	Court Officer Deed/Estate
       Con	Contract 15% Down payment regular terms
       ConLw	Contract Low Down payment and low interest
       ConLI	Contract Low Interest
       ConLD	Contract Low Down
       Oth	Other

In [ ]:
print("Número de missings:", df["SaleType"].isnull().sum())
print("Valores únicos:", np.unique(df["SaleType"]))

Conclusión: variable de futuro, la quito

In [ ]:
del df["SaleType"]

**Variable SaleCondition:**

Condition of sale

       Normal	Normal Sale
       Abnorml	Abnormal Sale -  trade, foreclosure, short sale
       AdjLand	Adjoining Land Purchase
       Alloca	Allocation - two linked properties with separate deeds, typically condo with a garage unit	
       Family	Sale between family members
       Partial	Home was not completed when last assessed (associated with New Homes)

In [ ]:
print("Número de missings:", df["SaleCondition"].isnull().sum())
print("Valores únicos:", np.unique(df["SaleCondition"], return_counts=True))

Me quedo solo con ventas normales:

In [ ]:
df = df[df["SaleCondition"] == "Normal"]

In [ ]:
print(np.unique(df["SaleCondition"]))

Ahora la elimino:

In [ ]:
del df["SaleCondition"]

### 41. Otras columnas

**Variable MiscVal:**

       Value of miscellaneous feature

In [ ]:
print("Número de missing values:", df["MiscVal"].isnull().sum())

In [ ]:
# estadísticas para valores mayores que cero:
df.loc[df["MiscVal"]>0, ["MiscVal"]].describe().T

**Variable MiscFeature:**

Miscellaneous feature not covered in other categories

       Elev	Elevator
       Gar2	2nd Garage (if not described in garage section)
       Othr	Other
       Shed	Shed (over 100 SF)
       TenC	Tennis Court
       NA	None

In [ ]:
print("Número de missing values: {} (de {})".format(df["MiscFeature"].isnull().sum(),
                                                    len(df)))

Como hay muy pocos valores que no sean missing elimino esta variable

In [ ]:
del df["MiscFeature"]

## Estadísticas de la base de datos limpia antes de hacer get_dummies

In [ ]:
pd.set_option('display.max_rows', 500) # para que al visualizar imprima todas
pd.DataFrame(df.dtypes, columns=["Tipo"])

In [ ]:
print("Número de casas:", len(df))
print("Número de missing values:", df.isnull().sum().sum())

In [ ]:
print("Variables que son categóricas:\n")
print(df.dtypes[df.dtypes==object])

In [ ]:
for n in df.columns[df.dtypes==object]:
    print("- Valores únicos de variable {}:".format(n))
    print(np.unique(df[n]))
    print()

In [ ]:
df.shape

## Get_dummies y eliminación de columnas constantes

In [ ]:
df = pd.get_dummies(df)
df.shape

In [ ]:
# Quito las columnas constantes:
df = df.loc[:,df.std(axis=0)>0]
df.shape

In [ ]:
print(list(df.columns))

In [ ]:
df.describe().T

In [ ]:
# Grabar la versión limpia del dataset a fichero:
df.to_csv("./data/dataset_housing_preprocesado.csv", index=False)

## Visualización de algunas variables

In [ ]:
print("Ahora sacaremos algunas estadísticas de la base de datos")
print('\nLa mediana de "SalePrice" es', df['SalePrice'].median(axis = 0), "\n")
df[['SalePrice']].describe().T

In [ ]:
plt.figure(figsize=(10,4))
ax = sns.distplot(df['SalePrice'], kde=True, rug=True)
plt.show()
print("Skewness:", df['SalePrice'].skew().round(3))
print("Kurtosis:", df['SalePrice'].kurt().round(3))

In [ ]:
corr = df.corr()
sns.clustermap(corr, vmin=-1, vmax=1, square=True, cmap=sns.color_palette("RdBu_r", 21));

In [ ]:
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
sns.pairplot(df[cols], size = 2.5, kind='reg',
             plot_kws={'line_kws':{'color':'red'}, 'scatter_kws': {'alpha': 0.1}}
            );

### SalePrice vs OverallQual ###

In [ ]:
data = pd.concat([df['OverallQual'], df['SalePrice']], axis=1)
plt.figure(figsize=(6, 4))
fig = sns.boxplot(x='OverallQual', y="SalePrice", data=data)
fig.set_xlabel('OverallQual', fontsize=16)
fig.set_ylabel('SalePrice', fontsize=16)

In [ ]:
data = pd.concat([df['OverallQual']*df['OverallQual'], df['SalePrice']], axis=1)
plt.figure(figsize=(6, 4))
plt.plot(data['OverallQual'], data["SalePrice"], 'o', alpha=0.1)
fig.set_xlabel('OverallQual', fontsize=16)
fig.set_ylabel('SalePrice', fontsize=16);

In [ ]:
xx = np.linspace(0, 10, 100)
plt.plot(xx, xx*xx); # eje x: x. eje y: x*x
# y = x^2
# u = x^2
# y = u

### SalePrice vs YearBuilt ###

In [ ]:
plt.figure(figsize=(20, 6))
fig = sns.boxplot(x = 'YearBuilt', y = 'SalePrice', data = df)
fig.axis(ymin=0, ymax=800000)
plt.xticks(rotation=90)
fig.set_xlabel('YearBuilt', fontsize=16)
fig.set_ylabel('SalePrice', fontsize=16)
plt.show()

### Lista de atributos categóricos ###

In [ ]:
print(df.select_dtypes(include=['object']).columns.values)

### Lista de atributos numéricos ###

In [ ]:
print(df.select_dtypes(exclude=['object']).columns.values)

In [ ]:
print("\nRanking de atributos por el valor absoluto de su")
print("correlación con el target (SalePrice):\n")
pd.DataFrame((df.corr()["SalePrice"].drop("SalePrice").abs()).sort_values(ascending=False))